# ResearchOps Community Toolbox Census + Graph
## Demo I - Data Ingestion & Transformation
#### Author: Pete Tunkis
#### Date: 2024-10-29

This notebook provides replicable code that (1) connects to the Airtable source dataset and imports it as a pandas dataframe, and (2) transforms the source data into a set of independent node and relationship tables that will serve as reference for nodes and edges in the graph.

In [13]:
### Environment

# Libraries

import os
from dotenv import load_dotenv
from airtable import Airtable
import pandas as pd
import hashlib
import numpy as np

# Paths
data_dir = './data/'

# Initialization
load_dotenv()

# Parameters
AIRTABLE_TOKEN = os.getenv('TOKEN')
AIRTABLE_BASE_ID = os.getenv('BASE_ID')
AIRTABLE_TABLE_ID = os.getenv('TABLE_ID')
AIRTABLE_VIEW = os.getenv('VIEW')

### Ingestion
First, I make a call to the Airtable API using the id/key provided by the ResearchOps Community - this is an 'airtable' class object that needs to be opened up and get records pulled out. Once I have the airtable object in memory, I pull out records and convert them to a dataframe.

In [9]:
### Ingestion

# Call the API
airtable = Airtable(base_id = AIRTABLE_BASE_ID
                    , api_key = AIRTABLE_TOKEN)
print(airtable)

# Grab records
records = { "records": [] }
for r in airtable.iterate(table_name = AIRTABLE_TABLE_ID
                           , view = AIRTABLE_VIEW):
    records["records"].append(r)

# Convert records into a pandas DataFrame
raw_df = pd.DataFrame([r['fields'] for r in records['records']])
display(raw_df)

,Business,Industry,Company type,In which country (countries) do you and/or your team conduct research? (Where are your participants located?),"From which countries do you or your co-workers conduct research? (If you work with multiple teams, please list all of the countries where the researchers, or people who do research, are located.)",Number of researchers,Maturity,"Which of the following categories best describes your current area of expertise? (If you have many roles, choose the discipline you spend most of your time doing.)",ReOps main responsibilities,How long have you been working in user experience and/or product?,...,Concept testing,Tree testing,First click testing,Task analysis,UX Benchmarking,Unmoderated usability testing,Heuristic evaluation,Insights repository,Focus group,Eye tracking
0,[Start-up or small corporation],"[Tech, Medicine/Health/Wellness, Logistics & S...",[Business-to-business-to-customer (B2B2C)],[USA],[USA],2-5,0: Absence/Unawareness of UX Research,[User or design research],"[Generalist , Research libraries, Recruitment,...",7 - 10 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[Freelance/consulting/contractor/self-employed ],"[Tech, Telecommunications, Consumer Services]",[Business-to-customer (B2C)],[USA],[USA],NaN,0: Absence/Unawareness of UX Research,[Leadership (manager or director)],"[Generalist , Research libraries, Recruitment,...",7 - 10 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[Start-up or small corporation],[Tech],[Business-to-business (B2B)],NaN,NaN,NaN,0: Absence/Unawareness of UX Research,[Research operations],"[Recruitment, Training and mentoring]",Less than a year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[Government/public sector],[Design],[Public sector],[Australia],[Australia],21-50,0: Absence/Unawareness of UX Research,[User or design research],"[Generalist , Training and mentoring]",7 - 10 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[Start-up or small corporation],"[Tech, Civic & Social Organizations]",[Business-to-business (B2B)],[USA],"[USA, Germany, UK]",1,2: Adoption of UX research into projects,[UX or product design],NaN,7 - 10 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,[In-house design in a medium or large corporat...,"[Tech, Financial Services, Design]",[Business-to-business (B2B)],[USA],[USA],1,1: UX Research Awareness - Ad Hoc Research in ...,[User or design research],[Generalist ],1 - 3 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,[In-house design in a medium or large corporat...,"[Medicine/Health/Wellness, Research, Consumer ...",[Business-to-customer (B2C)],"[Austria, Germany, Netherlands]",[Global],NaN,1: UX Research Awareness - Ad Hoc Research in ...,[User or design research],[Generalist ],7 - 10 years,...,"[UserZoom GO, Optimal Workshop, Quantilope]",[Optimal Workshop],[Optimal Workshop],[UserZoom GO],[quantilope],[UserZoom GO],NaN,NaN,NaN,NaN
202,[Start-up or small corporation],"[Tech, Design, Research]",[Business-to-business (B2B)],[USA],[USA],6-10,1: UX Research Awareness - Ad Hoc Research in ...,[User or design research],"[Research libraries, Recruitment, Training and...",4 - 6 years,...,"[Loom, UserTesting]",NaN,NaN,"[UserTesting, Zoom]",NaN,[UserTesting],[Google Sheets],"[Confluence, Notion]",NaN,NaN
203,[Start-up or small corporation],"[Tech, Education]",[Business-to-customer (B2C)],[Europe],[USA],2-5,NaN,[User or design research],[Generalist ],More than 15 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Save a csv of the raw data as backup (do not save to git)
filename = 'raw_df.csv'
raw_df.to_csv(data_dir + filename, index = False)

### Data Munging (a.k.a. transforming the data to the right shapes)
With my data now ready in its raw form, there are number of cleanups and transformations that need to be made.

The first thing I do is create a copy of the raw dataframe that I will manipulate, and then since each record in the dataset comes from a unique respondent, I create a unique hash ID based on all values in that record that represent the given respondent. Lots of people use UUIDs instead of hash IDs, and there are lots of ways to approach generating unique IDs; this doesn't matter here, but it might in your project.

Below that, I rename columns to make them more easily referable (for me). I still want to save the old column names for later (especially the ones I renamed to `tool-q-xx`) for when I construct the node and relationship tables.

Finally, I convert NaNs into empty strings, since pretty much everything in the data is of string or 'object' type.

In [15]:
###### Data Munging

### Basics

# Start off by creating a copy of the dataframe
prepped_df = raw_df.copy()

# Create a unique hash ID to link each element of the data
prepped_df['respondent-id'] = [hashlib.md5(str(x).encode('utf-8')).hexdigest() for x in prepped_df.T.to_dict().values()]

### Next up, convert all the column names to something workable...

# Start with a dictionary mapping original column names to 'q_01' type approach
# This dictionary will come in handy later on
column_names_dict = {'Business': 'business'
                     , 'Industry': 'industry'
                     , 'Company type': 'company'
                     , 'In which country (countries) do you and/or your team conduct research? (Where are your participants located?)': 'country-participants'
                     , 'From which countries do you or your co-workers conduct research? (If you work with multiple teams, please list all of the countries where the researchers, or people who do research, are located.)': 'country-researchers'
                     , 'Number of researchers': 'num-researchers'
                     , 'Maturity': 'maturity'
                     , 'Which of the following categories best describes your current area of expertise? (If you have many roles, choose the discipline you spend most of your time doing.)': 'discipline'
                     , 'ReOps main responsibilities': 'responsibility'
                     , 'How long have you been working in user experience and/or product?': 'len-experience'
                     , 'Research roadmapping': 'tools-q-01'
                     , 'Research planning': 'tools-q-02'
                     , 'Project management': 'tools-q-03'
                     , 'Issue tracking': 'tools-q-04'
                     , 'Finding participants': 'tools-q-05'
                     , 'Contacting participants': 'tools-q-06'
                     , 'Tracking participants': 'tools-q-07'
                     , 'Informed consent': 'tools-q-08'
                     , 'Participant incentives': 'tools-q-09'
                     , 'Participant scheduling': 'tools-q-10'
                     , 'Customer/participant relationship management': 'tools-q-11'
                     , 'Intercept': 'tools-q-12'
                     , 'Collaborative brainstorming': 'tools-q-13'
                     , 'Diagramming and mapping': 'tools-q-14'
                     , 'Creating wireframes': 'tools-q-15'
                     , 'Survey administration': 'tools-q-16'
                     , 'Diary study': 'tools-q-17'
                     , 'Infographics/data visualization': 'tools-q-18'
                     , 'Storytelling & presentations': 'tools-q-19'
                     , 'Mockups': 'tools-q-20'
                     , 'Prototyping': 'tools-q-21'
                     , 'Moderated usability testing': 'tools-q-22'
                     , 'Note taking': 'tools-q-23'
                     , 'Session recording': 'tools-q-24'
                     , 'Video editing': 'tools-q-25'
                     , 'Sharing findings': 'tools-q-26'
                     , 'Research repository': 'tools-q-27'
                     , 'Participatory design': 'tools-q-28'
                     , 'Card sorting': 'tools-q-29'
                     , 'Accessibility evaluation': 'tools-q-30'
                     , 'Competitive analysis': 'tools-q-31'
                     , 'User interviews': 'tools-q-32'
                     , 'Transcription': 'tools-q-33'
                     , 'Text tagging': 'tools-q-34'
                     , 'Path analysis': 'tools-q-35'
                     , 'Sentiment analysis': 'tools-q-36'
                     , 'Heat map': 'tools-q-37'
                     , 'A/B testing': 'tools-q-38'
                     , 'Search-log analysis': 'tools-q-39'
                     , 'Concept testing': 'tools-q-40'
                     , 'Tree testing': 'tools-q-41'
                     , 'First click testing': 'tools-q-42'
                     , 'Task analysis': 'tools-q-43'
                     , 'UX Benchmarking': 'tools-q-44'
                     , 'Unmoderated usability testing': 'tools-q-45'
                     , 'Heuristic evaluation': 'tools-q-46'
                     , 'Insights repository': 'tools-q-47'
                     , 'Focus group': 'tools-q-48'
                     , 'Eye tracking': 'tools-q-49'
                     }

# Rename columns
prepped_df = prepped_df.rename(columns = column_names_dict)

# Change all nan to blank string
prepped_df = prepped_df.fillna('')

One last thing to do - I've chosen to loop some code, but not all (specifically when sending commands pushing data to the graph), but for those parts where I do want while loops, I created some lists of column-groups: those that only ever have a single value, those with multiple values; those that provide respondent demographics, and those that cover tools (i.e., the use-case columns).

Also note that I created a reversed dictionary to complement the column-name mapping from above for when I want to refer back to original column names *from* the new ones.

In [24]:
### Collect dictionaries/lists for looping

# Invert key-values above so the key is the new name, value is the "description"
column_dict = {v: k for k, v in column_names_dict.items()}

# List of single-value and multi-value columns, as well as the biographical and tools questions
single_val_cols = ['respondent-id', 'num-researchers', 'maturity', 'len-experience', 'discipline']
multi_val_cols = [x for x in prepped_df.columns if x not in single_val_cols]
bio_cols = [x for x in prepped_df.columns if '-q-' not in x]
tool_cols = [x for x in prepped_df.columns if '-q-' in x]

###############################################################################

# Take a look
display(prepped_df)

,business,industry,company,country-participants,country-researchers,num-researchers,maturity,discipline,responsibility,len-experience,...,tools-q-41,tools-q-42,tools-q-43,tools-q-44,tools-q-45,tools-q-46,tools-q-47,tools-q-48,tools-q-49,respondent-id
0,[Start-up or small corporation],"[Tech, Medicine/Health/Wellness, Logistics & S...",[Business-to-business-to-customer (B2B2C)],[USA],[USA],2-5,0: Absence/Unawareness of UX Research,[User or design research],"[Generalist , Research libraries, Recruitment,...",7 - 10 years,...,,,,,,,,,,a24c55c88c441c468f0216dd5a6f78f6
1,[Freelance/consulting/contractor/self-employed ],"[Tech, Telecommunications, Consumer Services]",[Business-to-customer (B2C)],[USA],[USA],,0: Absence/Unawareness of UX Research,[Leadership (manager or director)],"[Generalist , Research libraries, Recruitment,...",7 - 10 years,...,,,,,,,,,,e535ac9b4755d584d31f8a4e1cbee9fa
2,[Start-up or small corporation],[Tech],[Business-to-business (B2B)],,,,0: Absence/Unawareness of UX Research,[Research operations],"[Recruitment, Training and mentoring]",Less than a year,...,,,,,,,,,,9fe8f5ca1760999b4ba8dac18b274368
3,[Government/public sector],[Design],[Public sector],[Australia],[Australia],21-50,0: Absence/Unawareness of UX Research,[User or design research],"[Generalist , Training and mentoring]",7 - 10 years,...,,,,,,,,,,df0ac91d632f3b5b679a607006698cea
4,[Start-up or small corporation],"[Tech, Civic & Social Organizations]",[Business-to-business (B2B)],[USA],"[USA, Germany, UK]",1,2: Adoption of UX research into projects,[UX or product design],,7 - 10 years,...,,,,,,,,,,7f908caf1cbb61c0f6ce6dd09414a668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,[In-house design in a medium or large corporat...,"[Tech, Financial Services, Design]",[Business-to-business (B2B)],[USA],[USA],1,1: UX Research Awareness - Ad Hoc Research in ...,[User or design research],[Generalist ],1 - 3 years,...,,,,,,,,,,8e066588cc0d697f9064f544b068cf74
201,[In-house design in a medium or large corporat...,"[Medicine/Health/Wellness, Research, Consumer ...",[Business-to-customer (B2C)],"[Austria, Germany, Netherlands]",[Global],,1: UX Research Awareness - Ad Hoc Research in ...,[User or design research],[Generalist ],7 - 10 years,...,[Optimal Workshop],[Optimal Workshop],[UserZoom GO],[quantilope],[UserZoom GO],,,,,7f9d6496e84a1f6d423631217424f3d3
202,[Start-up or small corporation],"[Tech, Design, Research]",[Business-to-business (B2B)],[USA],[USA],6-10,1: UX Research Awareness - Ad Hoc Research in ...,[User or design research],"[Research libraries, Recruitment, Training and...",4 - 6 years,...,,,"[UserTesting, Zoom]",,[UserTesting],[Google Sheets],"[Confluence, Notion]",,,8bb219c43243221b092f1388b8cd44e2
203,[Start-up or small corporation],"[Tech, Education]",[Business-to-customer (B2C)],[Europe],[USA],2-5,,[User or design research],[Generalist ],More than 15 years,...,,,,,,,,,,5f85d6f6756c31daede53d9e6963724e


## Setting up the Graph (tables)

### Nodes
Based on the Toolbox data, the graph will consist of the following nodes:
- Respondent
- Business
- Company
- Discipline
- Industry
- Responsibility
- Location
- Tool

### Relationships
Based on the nodes listed above, the graph will also contain the following relationships (for now):
- IS_BUSINESS_TYPE
- IS_COMPANY_TYPE
- IS_DISCIPLINE
- IN_INDUSTRY
- HAS_RESPONSIBILITY
- HAS_PARTICIPANTS_IN
- HAS_RESEARCHERS_IN
- USES

### Relationship Tables
Whether you want to start with node or relationship tables depends on your data, and the complexity you find yourself addressing.

In this case, creating the relationship tables first made more sense, as I eventually refer to them when creating some of my node tables; however, I probably could have done this the other way around just as well...it really doesn't matter, at least not here.

Each relationship table's code is virtually identical to a point--I chose not to loop over all of this to exhaustively show each object's providence. 

In [17]:
###### Relationship Tables

### Business
business_rel = prepped_df[['respondent-id', 'business']].copy()
business_rel['business'] = business_rel['business'].str.replace(r'([˙\[\]\'\"])+', '', regex = True).str.split(',')
business_rel = business_rel.explode('business')
business_rel['business'] = business_rel['business'].str.strip()
business_rel = business_rel[business_rel['business'] != '']

### Industry
industry_rel = prepped_df[['respondent-id', 'industry']].copy()
industry_rel['industry'] = industry_rel['industry'].str.replace(r'([˙\[\]\'\"])+', '', regex = True).str.split(',')
industry_rel = industry_rel.explode('industry')
industry_rel['industry'] = industry_rel['industry'].str.strip()
industry_rel = industry_rel[industry_rel['industry'] != '']

### Company Type
company_rel = prepped_df[['respondent-id', 'company']].copy()
company_rel['company'] = company_rel['company'].str.replace(r'([˙\[\]\'\"])+', '', regex = True).str.split(',')
company_rel = company_rel.explode('company')
company_rel['company'] = company_rel['company'].str.strip()
company_rel = company_rel[company_rel['company'] != '']

### Responsibility
responsibility_rel = prepped_df[['respondent-id', 'responsibility']].copy()
responsibility_rel['responsibility'] = responsibility_rel['responsibility'].str.replace(r'([˙\[\]\'\"])+', '', regex = True).str.split(',')
responsibility_rel = responsibility_rel.explode('responsibility')
responsibility_rel['responsibility'] = responsibility_rel['responsibility'].str.strip()
responsibility_rel = responsibility_rel[responsibility_rel['responsibility'] != '']



Tools and locations are special - although they are respectively independent, how respondents connect to them varies: 

**Tools** may be used for *multiple use cases*, meaning multiple relationships could exist between one respondent and one tool. To differentiate, I refer back to the original column names for each tool's use-case column and set that string as a property for each tool relationship.

**Locations** are what they are--but we differentiate between whether a respondent has researchers in a location versus participants. Eventually we will create separate relationships to denote researcher/participant locations, but I also carry over the column from which locations are drawn as the relationship property-to-be.

In [18]:
### Tools
tool_rel = pd.DataFrame()

for col in tool_cols:
    df = prepped_df[['respondent-id', col]].copy()
    df[col] = df[col].str.replace(r'([˙\[\]\'\"])+', '', regex = True).str.split(',')
    df = df.explode(col)
    df[col] = df[col].str.strip()
    df = df[df[col] != '']
    df['purpose'] = col
    df = df.rename(columns = {col: 'tool'})
    tool_rel = pd.concat([tool_rel, df], ignore_index = True)

tool_rel['purpose'] = tool_rel['purpose'].map({value: key for key, value in column_names_dict.items()})

In [19]:
### Locations - Researchers & Participants
location_rel = pd.DataFrame()

for col in ['country-participants', 'country-researchers']:
    df = prepped_df[['respondent-id', col]].copy()
    df[col] = df[col].str.replace(r'([˙\[\]\'\"])+', '', regex = True).str.split(',')
    df = df.explode(col)
    df[col] = df[col].str.strip()
    df = df[df[col] != '']
    df['party'] = col
    df = df.rename(columns = {col: 'location'})
    location_rel = pd.concat([location_rel, df], ignore_index = True)

Before moving on to node tables, it turns out that all Locations found in the data (for either researchers/participants) could use a little bit of consolidation, at least for the sake of reducing a touch of sparsity for the demo. **Nota bene**: make sure that any data manipulation as below *makes sense* or *is acceptable to your stakeholder*! Certain decisions, such as respelling `Brasil` to `Brazil` or `Asia Pacific` to `Asia/Pacific` is one thing, but I also condense `England` and `Northern Ireland` to `UK`. This may seem harmless at first blush, but there may be a *real reason* why the one respondent who indicated those countries did so rather than generically listing the UK...

...but for this PoC, keep calm and carry on.

In [ ]:
### Ancillary Data Cleanup

# Here I manually consolidate country names to reduce some sparsity, and some 
#   renaming to match ISO-3166-2 standards (and codes).

#loc_list = np.sort(location_rel.location.unique()) # Short reference, can consolidate manually
location_rel['location'] = location_rel['location'].replace({'America': 'USA'
                                                             , 'Asia Pacific': 'Asia/Pacific'
                                                             , 'Brasil': 'Brazil'
                                                             , 'China (Hong Kong)': 'Hong Kong'
                                                             , 'Czech Republic': 'Czechia'
                                                             , 'Emirates': 'United Arab Emirates'
                                                             , 'England': 'UK'
                                                             , 'Globally': 'Global'
                                                             , 'Korea': 'South Korea'
                                                             , 'Latin American': 'Latin America'
                                                             , 'NZ': 'New Zealand'
                                                             , 'Northern Ireland': 'UK'
                                                             , 'Turkey': 'Türkiye'
                                                             , 'UAE': 'United Arab Emirates'
                                                             , 'Vietnam': 'Viet Nam'})
    
###############################################################################



### Node Tables
I approached defining nodes and node properties in the same was as I did relationships, as seen below.

Before I begin creating the tables, I did a bit of consolidation to reduce sparsity around team-size, and also renamed missing data tied to size and length of experience for the purpose of crafting ordinal properties that will come in handy for dashboards/visualization later.

In [21]:
###### Node Tables

### Respondents
respondent_node = prepped_df[single_val_cols].copy()

for col in single_val_cols:
    respondent_node[col] = respondent_node[col].str.replace(r'([˙\[\]\'\"])+', '', regex = True)

# Consolidation
respondent_node['num-researchers'] = respondent_node['num-researchers'].replace({'100+': '50+'
                                                                                 , '': 'Undisclosed'})
respondent_node['len-experience'] = respondent_node['len-experience'].replace({'': 'Undisclosed'})

# Map ordinal vars for dashboard ordering - these will be properties
exp_map = {'I dont work in UX or product.': 0
           , 'Im still a student': 1
           , 'Less than a year': 2
           , '1 - 3 years': 3
           , '4 - 6 years': 4
           , '7 - 10 years': 5
           , '11 - 15 years': 6
           , 'More than 15 years': 7
           , 'Undisclosed': 8}
size_map = {'1': 0
            , '2-5': 1
            , '6-10': 2
            , '11-20': 3
            , '21-50': 4
            , '50+': 5
            , 'Undisclosed': 6}
respondent_node['exp-cat'] = respondent_node['len-experience'].map(exp_map)
respondent_node['size-cat'] = respondent_node['num-researchers'].map(size_map)


Most nodes' creation and property definitions are virtually identical - I leave these out of a loop to show each entity's creation explicitly.

In [ ]:
### Discipline (treat respondent_node as the rel table)
discipline_node = pd.DataFrame(columns = ['id', 'discipline'])
discipline_node['discipline'] = list(set(respondent_node[respondent_node['discipline'] != '']['discipline']))
discipline_node['id'] = [hashlib.md5(str(x).encode('utf-8')).hexdigest() for x in discipline_node['discipline'].T.to_dict().values()]

### Business
business_node = pd.DataFrame(columns = ['id', 'business'])
business_node['business'] = list(set(business_rel['business']))
business_node['id'] = [hashlib.md5(str(x).encode('utf-8')).hexdigest() for x in business_node['business'].T.to_dict().values()]

### Industry
industry_node = pd.DataFrame(columns = ['id', 'industry'])
industry_node['industry'] = list(set(industry_rel['industry']))
industry_node['id'] = [hashlib.md5(str(x).encode('utf-8')).hexdigest() for x in industry_node['industry'].T.to_dict().values()]

### Company
company_node = pd.DataFrame(columns = ['id', 'company'])
company_node['company'] = list(set(company_rel['company']))
company_node['id'] = [hashlib.md5(str(x).encode('utf-8')).hexdigest() for x in company_node['company'].T.to_dict().values()]

### Responsibility
responsibility_node = pd.DataFrame(columns = ['id', 'responsibility'])
responsibility_node['responsibility'] = list(set(responsibility_rel['responsibility']))
responsibility_node['id'] = [hashlib.md5(str(x).encode('utf-8')).hexdigest() for x in responsibility_node['responsibility'].T.to_dict().values()]

### Tool
tool_node = pd.DataFrame(columns = ['id', 'tool'])
tool_node['tool'] = list(set(tool_rel['tool']))
tool_node['id'] = [hashlib.md5(str(x).encode('utf-8')).hexdigest() for x in tool_node['tool'].T.to_dict().values()]

Locations aren't different from the other node tables in their own right, perhaps, but I also add an extra property with ISO country codes for any map-based visualizations/widgets that might take those data. In this case, ISO country codes may be used for Choropleth maps in NeoDash.

In [ ]:


### Location
location_node = pd.DataFrame(columns = ['id', 'location'])
location_node['location'] = list(set(location_rel['location']))
location_node['id'] = [hashlib.md5(str(x).encode('utf-8')).hexdigest() for x in location_node['location'].T.to_dict().values()]
# Add country codes where possible for visualizations
country_codes_df = pd.read_csv(data_dir + 'country_codes.csv', encoding = 'ISO-8859-1')
location_node = pd.merge(left = location_node
                         , right = country_codes_df
                         , how = 'left'
                         , left_on = 'location'
                         , right_on = 'location')